In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

import sys
print(sys.version)

3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:57:19) [GCC 11.3.0]


In [2]:
# Establish snowflake connection
conn = snowflake.connector.connect(
    account = "carelon-edaprod1.privatelink",
    user = "HAID",
    password = "PASSWORD",
    warehouse = "DL_AIFS_STAR_USER_WH_L",
    database = "NON_CRTFD_AIFS",
    schema = "DL_TS_STAR"
)

# Create a cursor object
cursor = conn.cursor()

In [3]:
#input training table
query="""select * from NON_CRTFD_AIFS.DL_TS_STAR.ihan_high_risk_preg_mother_final_data_iter_4_v1_training"""

In [4]:
df=pd.read_sql_query(query,conn)
print(df.shape)
df.head()

/home/jovyan/tmp/ipykernel_26166/2455608665.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(query,conn)


(356891, 52)


,MCID,BABY_BIRTH_DT,MOM_AGE,TARGET,HIGH_RISK_PROC_CD_CNT,TRANSVAGINAL_SONOGRAMS,VASA_PRIVIA,ANATOMICAL_FETAL_SURVEY,MLTPL_GEST,EPILEPSY,...,BPP_PROFILE,PAID_3_MONTHS,DOS1,DATLIST1,DOS2,DATLIST2,DOS3,DATLIST3,DOS4,DATLIST4
0,74691CE02F87A4D75F61272B93286993,2022-12-01,32,1,4,1,0,1,1,0,...,0,11201.24,"['2021-08-14','2022-04-08','2022-05-12','2022-...","[[22068],[6744,13769,14111],[10614,22166],[221...","['2021-08-14','2022-04-08','2022-05-12','2022-...","[[8507,8509],[1559,5856],[3379,4019,4020,4142,...","['2022-08-04','2022-11-30']","[[174,217],[20,47,85,94,103,116,117,118,121,12...",[],[]
1,93D9B8EAC3272EE1E85ACDE4544FE250,2022-11-14,34,0,5,0,1,1,1,0,...,1,17333.16,"['2021-07-28','2021-12-06','2021-12-07','2022-...","[[21887,22694],[10942],[10531,10745,11480,1184...","['2021-07-28','2021-12-06','2021-12-07','2022-...","[[5949],[4299,4314,4614],[1718,3379,4299,5860]...","['2021-12-06','2021-12-07','2022-04-27','2022-...","[[116],[116],[116],[116],[116],[116],[18,85,11...",[],[]
2,C1BB54BFB3AFA1DDEC3956DDF3B85AAE,2023-11-28,33,0,7,1,1,1,1,0,...,0,67558.09,"['2022-04-08','2022-04-14','2022-04-15','2022-...","[[14125],[11152,11602,22008,22152],[1401,11603...","['2022-04-08','2022-04-14','2022-04-15','2022-...","[[5859],[3373,5635,5862,5964],[3382,3383,3384,...","['2022-04-20','2022-05-05','2023-05-08','2023-...","[[174,251],[85,103,116,118,121,122,127,151,154...",[],[]
3,BA078AFF93F74487BF5FE7210EA0BEF2,2022-06-14,32,0,3,0,0,1,1,0,...,0,17322.57,"['2022-01-13','2022-02-04','2022-03-21','2022-...","[[22179,22265],[22186,22192,22265],[22086,2227...","['2022-01-13','2022-02-04','2022-03-21','2022-...","[[3679,3943],[3371],[3679,4080,4264,4266,4315,...","['2022-02-04','2022-06-14']","[[174],[20,85,93,116,118,121,154,266,279]]",[],[]
4,4B7D3E2701E886A3244298C18A4A4148,2023-08-05,42,0,9,1,0,1,1,1,...,1,21460.63,"['2023-01-06','2023-02-01','2023-04-12','2023-...","[[10402,10560,22167],[22184],[22199,22269],[22...","['2023-01-06','2023-02-01','2023-04-12','2023-...","[[1718,3379,3559,4102,4590,4653,4654,4668,4763...","['2023-05-09','2023-08-03']","[[10,20,85,103,116,117,121,122,123,174,283],[1...",[],[]


In [5]:
df['TARGET'].value_counts()

TARGET
0    341449
1     15442
Name: count, dtype: int64

## Balancing the data using balance ratio

The input data has an imbalance between the target classes (0 and 1). To address this, the data is balanced using a specified balance ratio.

In [6]:
# The input data is divided into two groups based on the TARGET column
df_tgt1=df[df['TARGET']==1]
df_tgt0=df[df['TARGET']==0]


The balance ratio is set to 0.05 (5%), which means the number of target 0 samples should be 95% of the target 1 samples.

The required number of target 0 samples to achieve the 95% balance ratio is calculated as follows:

This ensures that the ratio of target 0 to target 1 samples is approximately 19:1.

In [7]:
#balance_ratio 
balance=0.05 ##5%
balanceNumber = int(len(df_tgt1)*((1-balance)/balance)) 
print(balanceNumber)

293397


If the calculated balanceNumber exceeds the actual number of target 0 samples, the balanceNumber is set to the length of df_tgt0 to use all available target 0 samples.

In [8]:
if balanceNumber > len(df_tgt0):
    balanceNumber = len(df_tgt0)

print(balanceNumber)

293397


In [9]:
df = df_tgt0.sample(frac = 1)  
df = df.iloc[0:balanceNumber]   
df = pd.concat([df, df_tgt1])     
df = df.sample(frac=1)

In [10]:
df.shape

(308839, 52)

The balanced data is split into training, validation, and test sets using the returnTrainValTest function.

The function takes the input data and the desired proportions for training (75%), validation (25%), and test (0%) sets.
The training and validation sets are saved as CSV files.

In [11]:
trainProp=.75
valProp=.25
testProp=0

In [12]:
# For ihan sequential codes
def returnTrainValTest(inputDF,trainFrac, valFrac, testFrac):
    trainLen = int(len(inputDF) * trainFrac)
    valLen = int(len(inputDF)*(trainFrac+valFrac))
 
    if trainFrac + valFrac + testFrac != 1:
        raise Exception("The proportions of train, val, and test do not add up to 1")
    
   
    train_rtn = inputDF.iloc[0:trainLen]
    val_rtn = inputDF.iloc[trainLen:valLen]
    test_rtn = inputDF.iloc[valLen:]
    return train_rtn, val_rtn, test_rtn

In [13]:
train_data, val_data, test_data = returnTrainValTest(df,trainProp, valProp, testProp)

In [14]:
train_data.to_csv('train_data_iter4_v1.csv',index=False)
val_data.to_csv('val_data_iter4_v1.csv',index=False)
#test_data.to_csv('test_data.csv',index=False)

In [15]:
train_data.shape,val_data.shape

((231629, 52), (77210, 52))

In [16]:
df.shape

(308839, 52)

##  Saving OOT DATA 

In [17]:
#OOT table
query="""select * from NON_CRTFD_AIFS.DL_TS_STAR.ihan_high_risk_preg_mother_final_data_iter_4_v1_OOT"""

In [18]:
df_test=pd.read_sql_query(query,conn)

/home/jovyan/tmp/ipykernel_26166/1665930981.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test=pd.read_sql_query(query,conn)


In [19]:
df_test.head()

,MCID,BABY_BIRTH_DT,MOM_AGE,TARGET,HIGH_RISK_PROC_CD_CNT,TRANSVAGINAL_SONOGRAMS,VASA_PRIVIA,ANATOMICAL_FETAL_SURVEY,MLTPL_GEST,EPILEPSY,...,BPP_PROFILE,PAID_3_MONTHS,DOS1,DATLIST1,DOS2,DATLIST2,DOS3,DATLIST3,DOS4,DATLIST4
0,4D48957A617609FB57489210ADD43316,2024-01-26,28,0,6,1,1,1,1,0,...,1,19142.77,"['2023-06-12','2023-06-27','2023-07-11','2023-...","[[22167],[22186,22187,22248],[10595,22191,2220...","['2023-06-12','2023-06-27','2023-07-11','2023-...","[[3379,3553,3554,3684,4102,4107,4266,4542,4543...","['2023-09-05','2023-09-28','2023-11-07','2023-...","[[174],[174],[174],[85,117,122,123,197],[174],...",[],[]
1,435EB28E5772CB4C656A4F5ABEC50956,2024-08-07,28,0,5,1,0,1,1,1,...,0,6307.05,"['2022-12-29','2023-01-02','2023-01-25','2023-...","[[3182],[2797,2811,2852],[2301,3181,22541,2265...","['2022-12-29','2023-01-02','2023-01-25','2023-...","[[3105,6416],[5115,5861],[383,1440,1486,1489,5...","['2024-02-12','2024-08-06']","[[97,117,121,124,197,266],[281]]",[],[]
2,CA0C291B44176CA6153301F19D61F292,2024-04-02,37,0,3,1,0,1,1,0,...,1,38306.58,"['2023-01-03','2023-09-27','2023-11-27','2023-...","[[13796,14060,14125,22067],[22210,22260],[1080...","['2023-01-03','2023-09-27','2023-11-27','2023-...","[[1718,3550,4171,4278,4314,4813,5821,5894,5895...","['2023-01-03','2023-09-27','2023-11-27','2023-...","[[85,116,117,121,122,197,266],[174],[174],[93,...",[],[]
3,F3E5FA90C0340EAEB0BE5A8125689B24,2024-03-21,28,0,2,1,0,1,1,0,...,0,7742.72,"['2023-02-08','2023-02-14','2023-02-27','2023-...","[[10632,10640,21887,21969,22530],[10632],[7007...","['2023-02-08','2023-02-14','2023-02-27','2023-...","[[4645,4687,4763,4802,4807,4823,4875,5860,5942...","['2023-02-14','2023-06-29','2023-10-25','2024-...","[[174],[85,116,125,154,276,288],[174],[182,186...",[],[]
4,83E7A9AFC25B31179206F53E381D129B,2024-06-17,26,0,3,0,1,1,1,1,...,1,20240.49,"['2024-04-02','2024-04-03','2024-04-05','2024-...","[[11856,12732,13838,21143,22275],[10834,12732,...","['2024-04-02','2024-04-03','2024-04-05','2024-...","[[3378,4314,4315,4360,4382,4393,4604,4613,4614...","['2024-04-02','2024-05-14','2024-05-29','2024-...","[[85,118,121,174,197,267,291],[174],[85,116,11...","['2024-03-11','2024-04-06','2024-05-16']","[[54],[54],[54]]"


In [20]:
df_test.shape

(131602, 52)

In [21]:
df_test.to_csv('test_oot_iter_4_v1.csv')

## Additional Splits on OOT test data 

In [22]:
df_test['BABY_BIRTH_DT']=pd.to_datetime(df_test['BABY_BIRTH_DT'])
#test_data['BABY_DOB']==1 ===> baby's born before June 2024 .0 means baby's born after jun2024 to sep 2024
df_test['BABY_DOB']=(df_test['BABY_BIRTH_DT'] <= pd.to_datetime('2024-06-30', format='%Y-%m-%d')).astype(int)

In [23]:
#baby's born between jan to jun 2024
df_test_1=df_test[df_test['BABY_DOB']==1]
df_test_1.shape

(87437, 53)

In [24]:
df_test_1.to_csv('test_oot_iter_4_v1_6mnth.csv',index=False)

In [25]:
#baby's born between JUL to SEP 2024
df_test_2=df_test[df_test['BABY_DOB']==0]
df_test_2.shape

(44165, 53)

In [26]:
df_test_2.to_csv('test_oot_iter_4_v1_3mnth.csv',index=False)

## dictionary index table with description

In [27]:
query="""select * from NON_CRTFD_AIFS.DL_TS_STAR.ihan_high_risk_preg_mother_medical_code_idx_final_iter_4_v1_training"""

In [28]:
df_dict=pd.read_sql_query(query,conn)

/home/jovyan/tmp/ipykernel_26166/2567475494.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_dict=pd.read_sql_query(query,conn)


In [29]:
df_dict.head()

,IDX,CD_TYPE,CD_VALUE,DESCRIPTION
0,8914,DIAG_CD,M4722,"Other spondylosis with radiculopathy, cervical..."
1,2646,HLTH_SRVC_CD,59620,"Cesarean delivery only, following attempted va..."
2,10039,DIAG_CD,M9906,Segmental and somatic dysfunction of lower ext...
3,4772,HLTH_SRVC_CD,87502,Infectious agent detection by nucleic acid (DN...
4,5485,DIAG_CD,H903,"Sensorineural hearing loss, bilateral"


In [30]:
df_dict.to_csv('dict_indx_desc.csv',index=False)